In [ ]:
import boto3
import os
import io
import numpy as np
import pandas as pd
from datetime import datetime
import sagemaker
from sagemaker import RandomCutForest
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import RandomCutForest
from sagemaker import Model
from time import gmtime, strftime
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region_name = boto3.Session().region_name

In [ ]:
#aws_access_key_id = os.environ['AWS_ACCESS_KEY_ID'] = ""
#aws_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY'] = ""
#region_name = os.environ['AWS_DEFAULT_REGION'] = ""
#container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "randomcutforest", "latest")

# rcf models

In [ ]:
def rcf(base_job_name=str,\
        s3_train_data=str,\
        output_path=str,\
        train_instance_count=int,\
        train_instance_type=str,\
        num_samples_per_tree=int,\
        num_trees=int,\
        feature_dim=int):
    
    sagemaker_session = sagemaker.Session()
    role = get_execution_role()
    container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "randomcutforest", "latest")
    
    rcf_model = sagemaker.estimator.Estimator(
        container,
        role=role,
        output_path=output_path,
        train_instance_count=train_instance_count,
        train_instance_type=train_instance_type,
        sagemaker_session=sagemaker_session,
        base_job_name=base_job_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
    
    rcf_model.set_hyperparameters(
        num_samples_per_tree=num_samples_per_tree,
        num_trees=num_trees,
        feature_dim=feature_dim)
    
    s3_train_input = sagemaker.session.s3_input(
        s3_train_data,
        distribution='ShardedByS3Key',
        content_type='text/csv;label_size=0')
    
    return rcf_model.fit({'train': s3_train_input})

# rcf batch transform

In [ ]:
def rcf_transform(model_data=str,\
                  instance_count=int,\
                  instance_type=str,\
                  output_path=str,\
                  input_location=str,\
                  output_filter=str,\
                  job_name=str):
    
    model = Model(model_data=model_data,
                  image=sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "randomcutforest", "latest"),
                  role=get_execution_role())
    
    rcf_transformer = model.transformer(instance_count=instance_count,
                                        instance_type=instance_type,
                                        accept='text/csv',
                                        assemble_with='Line',
                                        output_path=output_path)
    
    rcf_transformer.transform(input_location,
                              output_filter=output_filter,
                              join_source='Input',
                              content_type='text/csv',
                              split_type='Line',
                              job_name=job_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
    
    rcf_transformer.wait()

# fog1

In [ ]:
fog1_df = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/fog1-sample-features.csv', header=None)

In [ ]:
fog1_df.head()

In [ ]:
fog1_df.shape

In [ ]:
del fog1_df

### fog1 training

In [ ]:
rcf('fog1-rcf-samples-1-min-granularity-',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/fog1-sample-features.csv',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/fog1/',\
    1,\
    'ml.m5.2xlarge',\
    512,\
    100,\
    168)

### fog1 inference

In [ ]:
rcf_transform('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/fog1/fog1-rcf-samples-1-min-granularity-2020-2020-04-15-20-16-52-238/output/model.tar.gz',\
              1,\
              'ml.m5.2xlarge',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/fog1/',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/fog1-sample-features.csv',\
              '$[0,-1]',\
              'fog1-rcf-results-1-min-granularity-')

### fog1 anomalies

In [ ]:
results = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/fog1/fog1-sample-features.csv.out', header=None)
results.columns = ['timestamp', 'score']
results['timestamp'] = pd.to_datetime(results['timestamp'], unit = 'ns')

In [ ]:
results.head()

In [ ]:
score_mean = results['score'].mean()
score_std = results['score'].std()
score_cutoff = score_mean + 3*score_std
anomalies = results[results['score'] > score_cutoff]

In [ ]:
results.count()

In [ ]:
anomalies.count()

# fog2

In [ ]:
fog2_df = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/fog2-sample-features.csv', header=None)

In [ ]:
fog2_df.head()

In [ ]:
fog2_df.shape

In [ ]:
del fog2_df

### fog2 training

In [ ]:
rcf('fog2-rcf-samples-1-min-granularity-',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/fog2-sample-features.csv',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/fog2/',\
    1,\
    'ml.m5.2xlarge',\
    512,\
    100,\
    168)

### fog2 inference

In [ ]:
rcf_transform('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/fog2/fog2-rcf-samples-1-min-granularity-2020-2020-04-15-20-23-53-411/output/model.tar.gz',\
              1,\
              'ml.m5.2xlarge',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/fog2/',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/fog2-sample-features.csv',\
              '$[0,-1]',\
              'fog2-rcf-results-1-min-granularity-')

### fog2 anomalies

In [ ]:
results = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/fog2/fog2-sample-features.csv.out', header=None)
results.columns = ['timestamp', 'score']
results['timestamp'] = pd.to_datetime(results['timestamp'], unit = 'ns')

In [ ]:
results.head()

In [ ]:
score_mean = results['score'].mean()
score_std = results['score'].std()
score_cutoff = score_mean + 3*score_std
anomalies = results[results['score'] > score_cutoff]

In [ ]:
results.count()

In [ ]:
anomalies.count()

# fad1

In [ ]:
fad1_df = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/fad1-sample-features.csv', header=None)

In [ ]:
fad1_df.head()

In [ ]:
fad1_df.shape

In [ ]:
del fad1_df

### fad1 training

In [ ]:
rcf('fad1-rcf-samples-1-min-granularity-',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/fad1-sample-features.csv',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/fad1/',\
    1,\
    'ml.m5.2xlarge',\
    512,\
    100,\
    174)

### fad1 inference

In [ ]:
rcf_transform('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/fad1/fad1-rcf-samples-1-min-granularity-2020-2020-04-15-20-48-08-196/output/model.tar.gz',\
              1,\
              'ml.m5.2xlarge',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/fad1/',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/fad1-sample-features.csv',\
              '$[0,-1]',\
              'fad1-rcf-results-1-min-granularity-')

### fad1 anomalies

In [ ]:
results = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/fad1/fad1-sample-features.csv.out', header=None)
results.columns = ['timestamp', 'score']
results['timestamp'] = pd.to_datetime(results['timestamp'], unit = 'ns')

In [ ]:
results.head()

In [ ]:
score_mean = results['score'].mean()
score_std = results['score'].std()
score_cutoff = score_mean + 3*score_std
anomalies = results[results['score'] > score_cutoff]

In [ ]:
results.count()

In [ ]:
anomalies.count()

# fad2

In [ ]:
fad2_df = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/fad2-sample-features.csv', header=None)

In [ ]:
fad2_df.shape

In [ ]:
del fad2_df

### fad2 training

In [ ]:
rcf('fad2-rcf-samples-1-min-granularity-',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/fad2-sample-features.csv',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/fad2/',\
    1,\
    'ml.m5.2xlarge',\
    512,\
    100,\
    226)

### fad2 inference

In [ ]:
rcf_transform('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/fad2/fad2-rcf-samples-1-min-granularity-2020-2020-04-15-20-57-20-735/output/model.tar.gz',\
              1,\
              'ml.m5.2xlarge',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/fad2/',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/fad2-sample-features.csv',\
              '$[0,-1]',\
              'fad2-rcf-results-1-min-granularity-')

### fad2 anomalies

In [ ]:
results = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/fad2/fad2-sample-features.csv.out', header=None)
results.columns = ['timestamp', 'score']
results['timestamp'] = pd.to_datetime(results['timestamp'], unit = 'ns')

In [ ]:
results.head()

In [ ]:
score_mean = results['score'].mean()
score_std = results['score'].std()
score_cutoff = score_mean + 3*score_std
anomalies = results[results['score'] > score_cutoff]

In [ ]:
results.count()

In [ ]:
anomalies.count()

# ac1

In [ ]:
ac1_df = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac1-sample-features.csv', header=None)

In [ ]:
ac1_df.head()

In [ ]:
ac1_df.shape

In [ ]:
del ac1_df

### ac1 training

In [ ]:
rcf('ac1-rcf-samples-1-min-granularity-',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac1-sample-features.csv',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/ac1/',\
    1,\
    'ml.m5.2xlarge',\
    512,\
    100,\
    6)

### ac1 inference

In [ ]:
rcf_transform('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/ac1/ac1-rcf-samples-1-min-granularity-2020--2020-04-15-21-07-59-567/output/model.tar.gz',\
              1,\
              'ml.m5.2xlarge',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/ac1/',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac1-sample-features.csv',\
              '$[0,-1]',\
              'ac1-rcf-results-1-min-granularity-')

### ac1 anomalies

In [ ]:
results = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/ac1/ac1-sample-features.csv.out', header=None)
results.columns = ['timestamp', 'score']
results['timestamp'] = pd.to_datetime(results['timestamp'], unit = 'ns')

In [ ]:
results.head()

In [ ]:
score_mean = results['score'].mean()
score_std = results['score'].std()
score_cutoff = score_mean + 3*score_std
anomalies = results[results['score'] > score_cutoff]

In [ ]:
results.count()

In [ ]:
anomalies.count()

# ac2

In [ ]:
ac2_df = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac2-sample-features.csv', header=None)

In [ ]:
ac2_df.head()

In [ ]:
ac2_df.shape

### ac2 training

In [ ]:
rcf('ac2-rcf-samples-1-min-granularity-',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac2-sample-features.csv',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/ac2/',\
    1,\
    'ml.m5.2xlarge',\
    512,\
    100,\
    8)

### ac2 inference

In [ ]:
rcf_transform('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/ac2/ac2-rcf-samples-1-min-granularity-2020--2020-04-15-21-23-09-215/output/model.tar.gz',\
              1,\
              'ml.m5.2xlarge',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/ac2/',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac2-sample-features.csv',\
              '$[0,-1]',\
              'ac2-rcf-results-1-min-granularity-')

### ac2 anomalies

In [ ]:
results = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/ac2/ac2-sample-features.csv.out', header=None)
results.columns = ['timestamp', 'score']
results['timestamp'] = pd.to_datetime(results['timestamp'], unit = 'ns')

In [ ]:
results.head()

In [ ]:
score_mean = results['score'].mean()
score_std = results['score'].std()
score_cutoff = score_mean + 3*score_std
anomalies = results[results['score'] > score_cutoff]

In [ ]:
results.count()

In [ ]:
anomalies.count()

# ac3

In [ ]:
ac3_df = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac3-sample-features.csv', header=None)

In [ ]:
ac3_df.head()

In [ ]:
ac3_df.shape

In [ ]:
del ac3_df

### ac3 training

In [ ]:
rcf('ac3-rcf-samples-1-min-granularity-',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac3-sample-features.csv',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/ac3/',\
    1,\
    'ml.m5.2xlarge',\
    512,\
    100,\
    19)

### ac3 inference

In [ ]:
rcf_transform('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/ac3/ac3-rcf-samples-1-min-granularity-2020--2020-04-15-21-27-23-468/output/model.tar.gz',\
              1,\
              'ml.m5.2xlarge',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/ac3/',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac3-sample-features.csv',\
              '$[0,-1]',\
              'ac3-rcf-results-1-min-granularity-')

### ac3 anomalies

In [ ]:
results = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/ac3/ac3-sample-features.csv.out', header=None)
results.columns = ['timestamp', 'score']
results['timestamp'] = pd.to_datetime(results['timestamp'], unit = 'ns')

In [ ]:
results.head()

In [ ]:
score_mean = results['score'].mean()
score_std = results['score'].std()
score_cutoff = score_mean + 3*score_std
anomalies = results[results['score'] > score_cutoff]

In [ ]:
results.count()

In [ ]:
anomalies.count()

# ac4

In [ ]:
ac4_df = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac4-sample-features.csv', header=None)

In [ ]:
ac4_df.head()

In [ ]:
ac4_df.shape

In [ ]:
del ac4_df

### ac4 training

In [ ]:
rcf('ac4-rcf-samples-1-min-granularity-',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac4-sample-features.csv',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/ac4/',\
    1,\
    'ml.m5.2xlarge',\
    512,\
    100,\
    49)

### ac4 inference

In [ ]:
rcf_transform('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/ac4/ac4-rcf-samples-1-min-granularity-2020--2020-04-15-21-32-07-814/output/model.tar.gz',\
              1,\
              'ml.m5.2xlarge',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/ac4/',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac4-sample-features.csv',\
              '$[0,-1]',\
              'ac4-rcf-results-1-min-granularity-')

### ac4 anomalies

In [ ]:
results = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/ac4/ac4-sample-features.csv.out', header=None)
results.columns = ['timestamp', 'score']
results['timestamp'] = pd.to_datetime(results['timestamp'], unit = 'ns')

In [ ]:
results.head()

In [ ]:
score_mean = results['score'].mean()
score_std = results['score'].std()
score_cutoff = score_mean + 3*score_std
anomalies = results[results['score'] > score_cutoff]

In [ ]:
results.count()

In [ ]:
anomalies.count()

# ac5

In [ ]:
ac5_df = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac5-sample-features.csv', header=None)

In [ ]:
ac5_df.head()

In [ ]:
ac5_df.shape

In [ ]:
del ac5_df

### ac5 training

In [ ]:
rcf('ac5-rcf-samples-1-min-granularity-',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac5-sample-features.csv',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/ac5/',\
    1,\
    'ml.m5.2xlarge',\
    512,\
    100,\
    58)

### ac5 inference

In [ ]:
rcf_transform('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/ac5/ac5-rcf-samples-1-min-granularity-2020--2020-04-15-21-36-56-576/output/model.tar.gz',\
              1,\
              'ml.m5.2xlarge',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/ac5/',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/ac5-sample-features.csv',\
              '$[0,-1]',\
              'ac5-rcf-results-1-min-granularity-')

### ac5 anomalies

In [ ]:
results = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/ac5/ac5-sample-features.csv.out', header=None)
results.columns = ['timestamp', 'score']
results['timestamp'] = pd.to_datetime(results['timestamp'], unit = 'ns')

In [ ]:
results.head()

In [ ]:
score_mean = results['score'].mean()
score_std = results['score'].std()
score_cutoff = score_mean + 3*score_std
anomalies = results[results['score'] > score_cutoff]

In [ ]:
results.count()

In [ ]:
anomalies.count()

# dn

In [ ]:
dn_df = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/diamond-north-sample-features.csv', header=None)

In [ ]:
dn_df.head()

In [ ]:
dn_df.shape

In [ ]:
del dn_df

### diamond north training

In [ ]:
rcf('dn-rcf-samples-1-min-granularity-',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/diamond-north-sample-features.csv',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/diamond-north/',\
    1,\
    'ml.m5.2xlarge',\
    512,\
    100,\
    115)

### diamond north inference

In [ ]:
rcf_transform('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/diamond-north/dn-rcf-samples-1-min-granularity-2020-0-2020-04-15-21-51-01-666/output/model.tar.gz',\
              1,\
              'ml.m5.2xlarge',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/diamond-north/',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/diamond-north-sample-features.csv',\
              '$[0,-1]',\
              'dn-rcf-results-1-min-granularity-')

### diamond north anomalies

In [ ]:
results = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/diamond-north/diamond-north-sample-features.csv.out', header=None)
results.columns = ['timestamp', 'score']
results['timestamp'] = pd.to_datetime(results['timestamp'], unit = 'ns')

In [ ]:
results.head()

In [ ]:
score_mean = results['score'].mean()
score_std = results['score'].std()
score_cutoff = score_mean + 3*score_std
anomalies = results[results['score'] > score_cutoff]

In [ ]:
results.count()

In [ ]:
anomalies.count()

# ds

In [ ]:
ds_df = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/diamond-south-sample-features.csv', header=None)

In [ ]:
ds_df.head()

In [ ]:
ds_df.shape

In [ ]:
del ds_df

### diamond south training

In [ ]:
rcf('ds-rcf-samples-1-min-granularity-',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/diamond-south-sample-features.csv',\
    's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/diamond-south/',\
    1,\
    'ml.m5.2xlarge',\
    512,\
    100,\
    115)

### diamond south inference

In [ ]:
rcf_transform('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/artifacts/diamond-south/ds-rcf-samples-1-min-granularity-2020-0-2020-04-15-22-01-19-858/output/model.tar.gz',\
              1,\
              'ml.m5.2xlarge',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/diamond-south/',\
              's3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/gold/1min-samples/diamond-south-sample-features.csv',\
              '$[0,-1]',\
              'ds-rcf-results-1-min-granularity-')

### diamond south anomalies

In [ ]:
results = pd.read_csv('s3://aero-iiot/datasets/e363549/iiot-analytics-pipeline/results/diamond-south/diamond-south-sample-features.csv.out', header=None)
results.columns = ['timestamp', 'score']
results['timestamp'] = pd.to_datetime(results['timestamp'], unit = 'ns')

In [ ]:
results.head()

In [ ]:
score_mean = results['score'].mean()
score_std = results['score'].std()
score_cutoff = score_mean + 3*score_std
anomalies = results[results['score'] > score_cutoff]

In [ ]:
results.count()

In [ ]:
anomalies.count()